In [1]:
using Revise
using Clang, Clang_jll

In [2]:
sdk_dir = `xcrun --show-sdk-path` |> open |> readchomp |> String

"/Applications/Xcode.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX.sdk"

In [3]:
index = Index()

Index(Ptr{Nothing} @0x0000000141f74210, 0, 1)

In [4]:
tu = Clang.parse_header(index, "048_testa.h") 


TranslationUnit(Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x000000010340fc20, Index(Ptr{Nothing} @0x0000000141f74210, 0, 1))

In [5]:
cu = Clang.getTranslationUnitCursor(tu)

CLCursor (CLTranslationUnit) 048_testa.h

In [6]:
dumpobj(cu)

 CLTranslationUnit 048_testa.h
   CLFunctionDecl c()
   CLFunctionDecl b()
   CLStructDecl 
      CLFieldDecl a CLInt int a
      CLFieldDecl a CLInt int b
   CLTypedefDecl Bar
     CLStructDecl 
        CLFieldDecl a CLInt int a
        CLFieldDecl a CLInt int b


In [7]:
children(cu)

4-element Vector{CLCursor}:
 CLCursor (CLFunctionDecl) c()
 CLCursor (CLFunctionDecl) b()
 CLCursor (CLStructDecl) 
 CLCursor (CLTypedefDecl) Bar

Precompiled Headers
=======================
- Try creating via clang and then loading via libclang (to work around module building problems) 
   

In [8]:
# Simple c header 
run(`$(clang()) -x c 048_testa.h -emit-ast -o testa.pch -isysroot $sdk_dir`);

In [9]:
ptr = Clang.clang_createTranslationUnit(index, "testa.pch")
## fails -- Using a custom built libclang but jll clang has version mismatch due to git hash being different -- need to lldb to see this, there is flag that disables this
## level of version compatibility checking but not sure how to pass it to libclang

Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x0000000000000000

In [11]:
myclang = "/Users/nehal/src/exploration/llvm-project/llvm/cmake-build-debug/bin/clang"
run(`$(myclang) -x c 048_testa.h -emit-ast -o testa.pch -isysroot $sdk_dir`);

In [12]:
ptr = Clang.clang_createTranslationUnit(index, "testa.pch")

Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x000000017753f610

In [ ]:
## works!

In [13]:
tu = parse_ast(index, "testa.pch")

TranslationUnit(Ptr{Clang.LibClang.CXTranslationUnitImpl} @0x00000001776d6ec0, Index(Ptr{Nothing} @0x0000000141f74210, 0, 1))

In [14]:
for c in tu |> Clang.getTranslationUnitCursor |> children
    println("$(c)\t$(get_filename(c))")
end

CLCursor (CLFunctionDecl) c()	/Users/nehal/src/ObjectiveC.jl/exploration/048_testc.h
CLCursor (CLFunctionDecl) b()	/Users/nehal/src/ObjectiveC.jl/exploration/048_testb.h
CLCursor (CLStructDecl) 	/Users/nehal/src/ObjectiveC.jl/exploration/048_testa.h
CLCursor (CLTypedefDecl) Bar	/Users/nehal/src/ObjectiveC.jl/exploration/048_testa.h
